Let's say we want to prepare data and try some scalers and classifiers for prediction in a classification problem. We will tune paramaters of classifiers by grid search technique.

Data preparing:

In [1]:
from sklearn.datasets import make_classification


X, y = make_classification()

Setting steps for our pipelines and parameters for grid search:

In [2]:
from reskit.core import Pipeliner


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC())]

scalers = [('standard', StandardScaler()),
           ('minmax', MinMaxScaler())]

steps = [('scaler', scalers),
         ('classifier', classifiers)]

param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}}

Creating a plan of our research:

In [3]:
pipe = Pipeliner(steps, param_grid=param_grid)
pipe.plan_table

,scaler,classifier
0,standard,LR
1,standard,SVC
2,minmax,LR
3,minmax,SVC


To tune parameters of models and evaluate this models, run:

In [4]:
pipe.get_results(X, y, scoring=['roc_auc'])

Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,scaler,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,standard,LR,0.91125,0.00655892,{'penalty': 'l1'},0.911332,0.00654054,[ 0.90311419 0.91911765 0.91176471]
1,standard,SVC,0.949596,0.0164625,{'kernel': 'rbf'},0.949611,0.0165447,[ 0.94809689 0.97058824 0.93014706]
2,minmax,LR,0.951691,0.0287759,{'penalty': 'l1'},0.951413,0.0287867,[ 0.97923875 0.96323529 0.91176471]
3,minmax,SVC,0.943382,0.0145963,{'kernel': 'poly'},0.943195,0.0145496,[ 0.96193772 0.94117647 0.92647059]
